In [1]:
import transformers
import pandas as pd
import numpy as np
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
modelname = 'microsoft/phi-2'

In [ ]:
model.model.num_hidden_layers

In [4]:
tokenizer = AutoTokenizer.from_pretrained(modelname, cache_dir='/work/pi_dhruveshpate_umass_edu/rseetharaman_umass_edu/.cache')

In [9]:
prompt = "Answer based on the context. Context: Czech Republic national football team is a member of the FIFA. Question: Czech Republic national football team is a member of the"

In [10]:
inputs = tokenizer([prompt], return_tensors="pt")

In [11]:
outputs = model.generate(**inputs, max_new_tokens=4, return_dict_in_generate=True, output_scores=True)
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [12]:
input_length = inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.4f} | {np.exp(score.numpy()):.2%}")

| 28702 | FIFA     | -1.5461 | 21.31%
| 28723 | .        | -0.8775 | 41.58%
|    13 | 
        | -2.1048 | 12.19%
|    13 | 
        | -0.2555 | 77.45%


In [40]:
values, indices = torch.topk(torch.softmax(outputs.scores[0], dim=1)[0], 5)

In [41]:
results = [tokenizer.decode([idx]) for idx in indices]

In [7]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
from tqdm import tqdm
import jsonlines
from util.globals import DATA_DIR
from math import ceil
from experiments.causal_trace import ModelAndTokenizer
from experiments.causal_trace import (
    predict_token, predict_topk
)
from dsets import KnownsDataset
import argparse

torch.set_grad_enabled(False)


model_name = 'google/gemma-7b'

mt = ModelAndTokenizer(
    model_name,
    low_cpu_mem_usage=False,
    torch_dtype=(torch.float16 if "20b" in model_name else None),
)

knowns = KnownsDataset(known_loc="data_mistral.jsonl")  # Dataset of known facts
prompts = [k['prompt'] for k in knowns][:10]


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded dataset with 922 elements


In [2]:
tokens, probs = predict_topk(mt, prompts)

In [3]:
tokens

[[' Apple', ' Beats', ' the', ' Dr', ' a'],
 [' Amazon', ' the', ' amazon', ' ', ' a'],
 [' CBS', ' September', ' Thursday', ' ', ' Monday'],
 [' the', ' a', ' Mac', ' Apple', ' an'],
 [' Italy', ' both', ' the', ' Italian', ' San'],
 [' Europe', ' ', ' Eurasia', ' the', ' Asia'],
 [' English', ' Marshall', ' the', ' both', ' Mic'],
 [' CBS', ' the', ' Via', ' a', ' which'],
 [' original', ' language', ' Latin', ' one', ' title'],
 [' composer', ' author', ' Italian', ' world', ' city']]